In [2]:
!pip install --upgrade tensorflow
!pip install transformers
!pip install keras
!pip install pandas
!pip install numpy
# !pip install -q pyyaml h5py

import pandas as pd
import numpy as np
import re
import keras
import numpy as np
import tensorflow as tf
import math

from sklearn.utils import shuffle

# from keras.layers import LSTM, GRU
# from keras.layers.embeddings import Embedding
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Model
from keras.layers import Input, Dense, Bidirectional,Flatten
# from keras.layers import concatenate,Dropout
from transformers import BertTokenizer, TFBertModel,TFBertForSequenceClassification

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.2.0)
     |████████████████████████████████| 645kB 9.2MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
     |████████████████████████████████| 3.8MB 45.4MB/s 
     |████████████████████████████████| 1.1MB 47.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9d102632d7051b485376ef866473cdebd30b8c4c84abb23cc6d08e47ce8b7530
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Using TensorFlow backend.


In [1]:

from google.colab import drive
drive.mount("/content/drive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Given a natural language query $q$ and a set of (predicate, object) tuples pertaining to an entity, the model aims to predict the *importance* and *utility* ranks of the (predicate, object) tuples with respect to the query $q$.

We encode the query $q$ and the predicate $p$ into $d$-dimensional embedding vectors $h_q$ and $h_p$ using a BiLSTM. Then, the model predicts a score for by using a dot product of $h_q$ and $h_p$. i.e. $\Psi(q, p) = h_q^Th_p$

We minimize the MSE Loss between the predicted score and the target utility or importance score.

In [0]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.bert_layer = TFBertModel.from_pretrained('bert-base-uncased')
        self.dense_layer = tf.keras.layers.Dense(1, activation="sigmoid")
        ## 50 x 768

    def __call__(self, x_train):
        intermediate = self.bert_layer(x_train)
        output = self.dense_layer(intermediate[1])
        
        return output

In [0]:
## homogeneous encoding for all the queries,predicates, objects
maxLengthPadding = 50
# Note : maxLengthOfTokens = 48;

input_dictionary = {}

# df = pd.read_csv("/content/drive/My Drive/Independent Study/Modified_Data.csv")
# df = pd.read_csv("/content/drive/My Drive/Independent Study/URI_only.csv")
# df = pd.read_csv("/content/drive/My Drive/Independent Study/Complete_Data_With_Targets.csv")
# df = pd.read_csv("/content/drive/My Drive/Independent Study/URI_with_imp.csv")
df = pd.read_csv("/content/drive/My Drive/Independent Study/URI_with_targets.csv")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def getBertParameters(tokens):
    attn_mask = []
    seg_ids = []
    # pos_ids = []
    if(len(tokens)<maxLengthPadding):
        attn_mask = [1]*len(tokens) + [0]*(maxLengthPadding-len(tokens))
    else:
        attn_mask = [1]*maxLengthPadding
    
    segment = 0
    for x in tokens:
        seg_ids.append(segment)
        if(x=='[SEP]'):
            segment = 1
    seg_ids+=[0]*(maxLengthPadding-len(tokens))
    return attn_mask,seg_ids


In [0]:
## defining custom model parameters as per tensorflow2 guidelines

model = MyModel()

loss_obj = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam( learning_rate=0.00005)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanSquaredError(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanSquaredError(name='test_accuracy')


In [0]:
groupedDataset = df.groupby('query')
queries = []
for q,g in groupedDataset:
  queries.append(q)

queries = shuffle(queries)


In [53]:




## data length = 95
data_length = len(groupedDataset)
split_length = int(0.8*data_length)

queries_for_training = queries[:split_length]
queries_for_testing = queries[split_length:]


test_data =[]
count = 0

queryTrainList = []
predTrainList = []
objTrainList = []

y_train = []

for query,group in groupedDataset:
    if(query in queries_for_testing):
      test_data.append([query,group["pred"].values,group["obj"].values,group["imp"].values])
    else: 
      for i in range(len(group["pred"].values)):
        queryTrainList.append(query)
        predTrainList.append(group["pred"].values[i])
        objTrainList.append(group["obj"].values[i])
        y_train.append(group["imp"].values[i])
    count+=1

## To give a higher importance to a higher utility value
def normalizeRanks(rankArray):
  maximum = 2
  for i in range(len(rankArray)):
    rankArray[i]=rankArray[i]/maximum
  return rankArray


y_train = normalizeRanks(y_train)



queryTrainList,predTrainList,objTrainList,y_train = shuffle(queryTrainList,predTrainList,objTrainList,y_train)


print(y_train[0:10])    
print(len(queryTrainList),len(predTrainList),len(objTrainList),len(test_data),len(y_train),data_length)


[0.0, 1.0, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5]
1097 1097 1097 19 1097 95


In [54]:
batch_size = 16
num_batches = math.floor(len(queryTrainList)/batch_size)
EPOCHS = 20


def train_step(query,pred,obj,target):
    input_ids = []
    attn_mask = []
    segment_ids = []

    for k in range(len(query)):
          in_id = tokenizer.encode(query[k],pred[k]+" "+obj[k], add_special_tokens=True)  # Batch size 1
          attn,seg = getBertParameters(tokenizer.convert_ids_to_tokens(in_id))
          in_id += [0]*(maxLengthPadding-len(in_id))
          input_ids.append(in_id)
          attn_mask.append(attn)
          segment_ids.append(seg)
          
    loss = 0
    with tf.GradientTape() as tape:
        input_dictionary['input_ids'] = tf.convert_to_tensor(np.array(input_ids)) ## [None, :]
        input_dictionary['attention_mask'] = tf.convert_to_tensor(np.array(attn_mask))  ## [None, :]
        input_dictionary['token_type_ids'] = tf.convert_to_tensor(np.array(segment_ids))  ## [None, :]
        output = model(input_dictionary)
        loss = (target-output)**2
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # input()
    return loss
  
        
    
ndcg_epoch_array = {
    
}

## [CLS] + [Query] + [SEP] + [PRED + OBJ] +[SEP]
for j in range(EPOCHS):
    for i in range(num_batches-1):
        index = i*batch_size
        q = queryTrainList[index:index+batch_size]
        p = predTrainList[index:index+batch_size]
        o = objTrainList[index:index+batch_size]
        actual_y = np.array(y_train[index:index+batch_size]).reshape(batch_size,1)
        loss = train_step(q,p,o,actual_y)

    q = queryTrainList[num_batches*batch_size:]
    p = predTrainList[num_batches*batch_size:]
    o = objTrainList[num_batches*batch_size:]
    if(len(q)>0):
        actual_y = y_train[num_batches*batch_size:]
        actual_y = np.array(actual_y).reshape(len(actual_y),1)
        print(actual_y)
        
        loss = train_step(q,p,o,actual_y)
        print("Loss after epoch "+str(j)+" :", loss)   
    n5,n10 = test_step()

    print("n5,n10:",n5,n10)


    ndcg_epoch_array[j] = [n5,n10]



    


[[0. ]
 [0.5]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0.5]
 [1. ]
 [0.5]]
Loss after epoch 0 : tf.Tensor(
[[0.01046777]
 [0.0062552 ]
 [0.03520949]
 [0.30581668]
 [0.03121926]
 [0.01727354]
 [0.05679343]
 [0.00218958]
 [0.02441305]], shape=(9, 1), dtype=float32)
[2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
n5,n10: 0.8277502737893991 0.8539832714891658
[[0. ]
 [0.5]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0.5]
 [1. ]
 [0.5]]
Loss after epoch 1 : tf.Tensor(
[[6.8226797e-03]
 [1.0796040e-03]
 [1.9329825e-03]
 [3.4071097e-01]
 [2.4150020e-02]
 [3.0934403e-02]
 [1.3142760e-01]
 [2.9792663e-04]
 [2.6681462e-02]], shape=(9, 1), dtype=float32)
[2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2]
[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2]
n5,n10: 0.87788574494226 0.8981272203961784
[[0. ]
 [0.5]
 [0. ]
 [0. ]
 [0. ]
 [0. ]
 [0.5]
 [1. ]
 [0.5]]
Loss after epoch 2 : tf.Tensor(
[[1.5695360e-03]
 [1.9263295e-03]
 [7.1019805e-03]
 [1.4125182e-01]
 [1.5240103e-01]
 [2.3279499e-02]
 [1.610

In [55]:
ndcg_epoch_array


{0: [0.8277502737893991, 0.8539832714891658],
 1: [0.87788574494226, 0.8981272203961784],
 2: [0.889085452066925, 0.9055034774742943],
 3: [0.8779874770667968, 0.9048284050797816],
 4: [0.8717839073504172, 0.8979630805328477],
 5: [0.8566280681212103, 0.8832700771987593],
 6: [0.8390781753326506, 0.8748321524517114],
 7: [0.8664522257853488, 0.8944954444811369],
 8: [0.8623078107890673, 0.8898963963758205],
 9: [0.8935991874173155, 0.9092718528937019],
 10: [0.8592639955825111, 0.8830790777186778],
 11: [0.8690262393892575, 0.8897577142256277],
 12: [0.860806547814115, 0.8873771193762889],
 13: [0.8553308714215051, 0.8771429845741393],
 14: [0.8419030133663847, 0.8684671905589285],
 15: [0.8706392398884882, 0.89613450349768],
 16: [0.8802355114169, 0.8942004165882376],
 17: [0.8769840838701812, 0.8894277490431457],
 18: [0.8383861263361176, 0.8615554968996788],
 19: [0.8478862344999464, 0.8699431071753785]}

In [0]:
# ## grouping the dataset by query to get the ndcg scores
def test_step():
    predictedQueryRanks = []
    groundTruthRanks = []

    integerValuedQueryRanks = []

    count = 0

    for x_test in test_data:
        query = x_test[0]
        predPerQuery = list(x_test[1])
        objPerQuery = list(x_test[2])
        target = list(x_test[3])
        g_t = []
        i_ranks = []
        predictedRanksPerQuery = []
        for i in range(len(predPerQuery)):
            input_ids = tokenizer.encode(query,predPerQuery[i]+" "+objPerQuery[i], add_special_tokens=True)  # Batch size 1
            attn_mask,segment_id = getBertParameters(tokenizer.convert_ids_to_tokens(input_ids))
            input_ids += [0]*(maxLengthPadding-len(input_ids))
            input_dictionary['input_ids'] = tf.constant(input_ids)[None, :]
            input_dictionary['attention_mask'] = tf.constant(attn_mask)[None, :]
            input_dictionary['token_type_ids'] = tf.constant(segment_id)[None, :]
            output = model(input_dictionary)
            predictedRanksPerQuery.append(output.numpy().tolist()[0][0])
            i_ranks.append(round(output.numpy().tolist()[0][0]*2))
        integerValuedQueryRanks.append(i_ranks)

        groundTruthRanks.append(target) 
        predictedQueryRanks.append(predictedRanksPerQuery)
        count+=1
    print(groundTruthRanks[0])
    print(integerValuedQueryRanks[0])
    
    return compute_ndcg_scores(groundTruthRanks,integerValuedQueryRanks)

    # print(groundTruthRanks[0])
    # print(predictedQueryRanks[0])
    # print(integerValuedQueryRanks[0])

    # for x,y in zip(groundTruthRanks,integerValuedQueryRanks):
    #     print("GRound truth : ", x)
    #     print("Predicted : " ,y)
    #     print("----------------------------------------------------------x-----------------------------------------x---------------------------")
    

In [0]:

from sklearn.metrics import ndcg_score, dcg_score 

def compute_ndcg_scores(groundTruthRanks,integerValuedQueryRanks): 
    ndcg_scores_5 = []
    ndcg_scores_10 = []
    count = 0
    for x,y in zip(groundTruthRanks,integerValuedQueryRanks):
        
        
        if(len(x)>1):
            true_relevance = np.asarray([x]) 
            relevance_score = np.asarray([y]) 

            ndcg_scores_5.append(ndcg_score(true_relevance, relevance_score,k=5))
            ndcg_scores_10.append(ndcg_score(true_relevance, relevance_score,k=10))
        

    return (sum(ndcg_scores_5)/len(ndcg_scores_5)),(sum(ndcg_scores_10)/len(ndcg_scores_10))



   
 